In [1]:
import pandas as pd
from gensim import models, corpora
from gensim.parsing.preprocessing import preprocess_string, strip_tags, strip_punctuation
from gensim.parsing.preprocessing import remove_stopwords, stem_text, strip_non_alphanum, strip_multiple_whitespaces
from gensim.parsing.preprocessing import strip_short, strip_numeric
from gensim.parsing.preprocessing import STOPWORDS
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import multiprocessing as mp
import nltk
import pandas as pd
import numpy as np
import tensorflow as tf
from scipy import spatial
import parmap
import os
import swifter
from tqdm import tqdm
import sqlite3

/home/ubuntu/anaconda3/envs/econ/lib/python3.7/site-packages/dask/dataframe/utils.py:15: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
topics_index_to_name_map = {
    0: 'Agriculture, animals, food and rural affairs',
    1: 'Asylum, immigration and nationality',
    2: 'Business, industry and consumers',
    3: 'Communities and families',
    4: 'Crime, civil law, justice and rights',
    5: 'Culture, media and sport',
    6: 'Defence',
    7: 'Economy and finance',
    8: 'Education',
    9: 'Employment and training',
    10: 'Energy and environment',
    11: 'European Union',
    12: 'Health services and medicine',
    13: 'Housing and planning',
    14: 'International affairs',
    15: 'Parliament, government and politics',
    16: 'Science and technology',
    17: 'Social security and pensions',
    18: 'Social services',
    19: 'Transport',
    20: 'Others'
}
topics_name_to_index_map = {y:x for x,y in topics_index_to_name_map.items()}

def strip_short2(text):
    return strip_short(text, minsize=4)


def preprocess_text(text):
    FILTERS = [lambda x: x.lower(), strip_multiple_whitespaces, strip_tags, strip_punctuation,
                   strip_non_alphanum, strip_numeric, strip_short2]
    return preprocess_string(text, FILTERS)

def preprocess(topic):
    ret = []
    topic = topic.strip()
    
    if '|' in topic:
        topics = topic.split('|')
        t = topics[0]
        t = t.strip()
        return topics_name_to_index_map[t]
        
    return topics_name_to_index_map[topic]

In [128]:
year = 2018

In [129]:
df = pd.read_csv('./bert_partitions_54_{}.csv'.format(year))
# df = df.loc[df['Program Name'].isin(['BBC News at One', 'BBC News at Ten', 'BBC News at Six'])].reset_index(drop=True)
df = df.drop(['Unnamed: 0'], axis=1)

In [130]:
df['Date'] = pd.to_datetime(df['Date'])

In [131]:
doc2vec_model = Doc2Vec.load('../models/doc2vec/doc2vec_news_{}_no_Others'.format(2017))

In [132]:
df

,Source,Date,Program Name,Time,Duration,Transcript,partitioned_transcript
0,BBC1 London,2018-06-02,BBC News,02:05,235 mins,"but also gathering storm clouds, some further ...","[p] but also gathering storm clouds, some furt..."
1,BBC1 London,2018-06-02,BBC News,13:00,15 mins,CRIMEWATCH ROADSHOW LIVE AA0252740VA01NAT BRDO...,[p] CRIMEWATCH ROADSHOW LIVE AA0252740VA01NAT ...
2,BBC1 London,2018-06-02,BBC News,17:50,10 mins,You’re lying! At least this is the real thing....,[p] You’re lying! At least this is the real th...
3,BBC1 London,2018-06-02,BBC London News,18:00,5 mins,at times flawed. It is a test of temperament a...,[p] at times flawed. It is a test of temperame...
4,BBC1 London,2018-06-02,BBC News,22:10,20 mins,# Ochi zhguchie i... # 0h! You have to say tha...,[p] # Ochi zhguchie i... # 0h! You have to say...
...,...,...,...,...,...,...,...
2120,BBC1 London,2018-01-31,BBC London News,13:30,15 mins,The researchers say whales and dolphins are am...,[p] The researchers say whales and dolphins ar...
2121,BBC News 24,2018-01-31,BBC News at Six,18:00,30 mins,BBC? I was extremely happily mnning the Royal ...,[p] BBC? I was extremely happily mnning the Ro...
2122,BBC1 London,2018-01-31,BBC London News,18:30,30 mins,Another high street flooded thanks Another hig...,[p] Another high street flooded thanks Another...
2123,BBC1 London,2018-01-31,BBC News at Ten,22:00,30 mins,"Now back at home, she’s speaking for the first...","[p] Now back at home, she’s speaking for the f..."


In [133]:
partitions = []
for index, row in df.iterrows():
    partition_string = row['partitioned_transcript']
    partition_date = row['Date']
    partition_source = row['Program Name']
    all_partitions = partition_string.split('\n')
    for partition in all_partitions:
        t = None
        if partition.startswith('[p]') and partition.endswith('[/p]'):
            t = 'p'
        else:
            t = 's'
        partitions.append((index, partition_date, partition_source, partition[3:-4], t))
partition_df = pd.DataFrame(partitions, columns=['partition_id', 'date', 'source', 'transcript', 'type'])

In [134]:
mask = (partition_df['date'] >= '2018-7-1') & (partition_df['date'] <= '2018-12-31')
partition_df = partition_df.loc[mask]

In [126]:
partition_df

,partition_id,date,source,transcript,type
0,0,2017-09-02,BBC News,and she paid a price for it. # ..Love you... ...,p
1,0,2017-09-02,BBC News,Undercover: Britain’s Immigration Secrets. On...,s
2,0,2017-09-02,BBC News,Put your thumb vertically above her and make ...,p
3,0,2017-09-02,BBC News,"One and two and three. One, two, three. Yeah,...",s
4,0,2017-09-02,BBC News,As we step into the first weekend in Septembe...,p
...,...,...,...,...,...
225253,2090,2017-12-18,BBC London News,"In Lapland, a group of disappointed children ...",p
225254,2090,2017-12-18,BBC London News,APPLAUSE And on her first day working at a Ch...,p
225255,2090,2017-12-18,BBC London News,"Not quite up there with The LastJedi, but a n...",p
225256,2090,2017-12-18,BBC London News,Please welcome Joe Wilkinson!,s


In [37]:
partition_df['vector'] = partition_df.swifter.apply(lambda x: doc2vec_model.infer_vector(preprocess_text(x['transcript'])), axis=1)

In [ ]:
del df

In [ ]:
articles = pd.read_csv('../data/news_predictions/news_{}_predictions.csv'.format(year))

In [ ]:
articles = articles.dropna(subset=['transcript'])
articles = articles.loc[articles.top1_topic != 'Others']

In [ ]:
articles = articles.drop(['Unnamed: 0'], axis=1)
articles = articles.loc[articles.month.isin([1,2,3,4,5,6])]
articles

In [ ]:
vector = []
transcripts = articles.transcript.values

preprocessed_transcripts = parmap.map(preprocess_text, transcripts, pm_pbar=True)

In [ ]:
vector_transcripts = parmap.map(doc2vec_model.infer_vector, preprocessed_transcripts, pm_pbar=True)
articles['vector'] = vector_transcripts

In [ ]:
del preprocessed_transcripts

In [ ]:
import datetime
articles['date'] = articles.apply(lambda x: datetime.date(x.year, x.month, x.day), axis=1)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from datetime import datetime, timedelta
from semantic_text_similarity.models import WebBertSimilarity
web_model = WebBertSimilarity(device='cpu', batch_size=10) #defaults to GPU prediction

def predict(partition):
    dt = partition.date
    start_dt = dt - timedelta(days=2)
    end_dt = dt + timedelta(days=2)
    articles_window = articles.loc[(articles['date'] >= start_dt) & (articles['date'] <= end_dt)]
    print(len(articles_window))
    partition_vector = np.array(partition.vector)
    partition_vector = partition_vector.reshape(1, 100)
    articles_vector = articles_window.vector.values
    vec = [articles_vector[i] for i in range(len(articles_vector))]
    vec = np.array(vec)
    sim = cosine_similarity(partition_vector, vec)
    max_index = np.argmax(sim)
    
    return articles_window.iloc[max_index]['top1_topic']

def predict_with_bert(partition):
    dt = partition.date
    start_dt = dt - timedelta(days=2)
    end_dt = dt + timedelta(days=2)
    articles_window = articles.loc[(articles['date'] >= start_dt) & (articles['date'] <= end_dt)]
    partition_transcript = np.array(partition.transcript)
    articles_transcript = articles_window.transcript.values
    input_data = [(partition_transcript, a) for a in articles_transcript]
    print(input_data[0])
    sim = web_model.predict(input_data)
    max_index = np.argmax(sim)
    
    return articles_window.iloc[max_index]['top1_topic']

In [ ]:
predict(partition_df.iloc[4])

In [ ]:
partition_df.iloc[4].transcript

In [ ]:
preds = []

for index, row in tqdm(partition_df.iterrows(), total=len(partition_df)):
    preds.append(predict(row))
    
partition_df['prediction'] = preds

In [135]:
partition_df.to_csv('./partitions_Jul-Dec_54_{}.csv'.format(year))